# recommendation system

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import rdflib
from bs4 import BeautifulSoup
import urllib
import re
import shutil
import requests
import copy

In [5]:
movie_df = pd.read_csv('./movies.csv', usecols=['title','genres'])
movie_df.head()

,title,genres
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy


In [59]:
movie_title=[' '.join(x.split(' ')[:-1]) for x in movie_df['title'].tolist()]
movie_time=[x.split(' ')[-1][1:5] for x in movie_df['title'].tolist()]
movie_genres=[ x.split('|') for x in movie_df['genres'].tolist()]

In [60]:
for i in range(100):
    print(i,movie_title[i],movie_time[i],movie_genres[i])

0 Toy Story 1995 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
1 Jumanji 1995 ['Adventure', 'Children', 'Fantasy']
2 Grumpier Old Men 1995 ['Comedy', 'Romance']
3 Waiting to Exhale 1995 ['Comedy', 'Drama', 'Romance']
4 Father of the Bride Part II 1995 ['Comedy']
5 Heat 1995 ['Action', 'Crime', 'Thriller']
6 Sabrina 1995 ['Comedy', 'Romance']
7 Tom and Huck 1995 ['Adventure', 'Children']
8 Sudden Death 1995 ['Action']
9 GoldenEye 1995 ['Action', 'Adventure', 'Thriller']
10 American President, The 1995 ['Comedy', 'Drama', 'Romance']
11 Dracula: Dead and Loving It 1995 ['Comedy', 'Horror']
12 Balto 1995 ['Adventure', 'Animation', 'Children']
13 Nixon 1995 ['Drama']
14 Cutthroat Island 1995 ['Action', 'Adventure', 'Romance']
15 Casino 1995 ['Crime', 'Drama']
16 Sense and Sensibility 1995 ['Drama', 'Romance']
17 Four Rooms 1995 ['Comedy']
18 Ace Ventura: When Nature Calls 1995 ['Comedy']
19 Money Train 1995 ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']
20 Get Shorty 1995 

In [61]:
test=['Toy Story','Jumanji']

In [62]:
def check_genres_in_list(genres_1,genres_2):
    for g in genres_1:
        if g in genres_2:
            return 1
    return 0


def recommend_movies(test,n):
    number_of_all_movies=len(movie_title)

    movie_index=[]
    for t in test:
        try:
            movie_index.append(movie_title.index(t))
        except:
            print('movie has not been found')

    if movie_index==[]:
        print('it seems like you did not input even a movie, it is hard for me to recommmend.')
    else:
        movie_r_time=[]
        movie_r_genres=[]
        for i in movie_index:
            movie_r_time.append(movie_time[i])
            movie_r_genres.append(movie_genres[i])

    genres_counter=Counter()
    for l in movie_r_genres:
        for item in l:
            genres_counter.update({item:1})


    genres_counter=dict(sorted(genres_counter.items(), key=lambda item: item[1],reverse=True))
    movie_r_time=list(set(movie_r_time))

    # print(movie_r_time)
    # print(genres_counter)

    time_r_list=movie_r_time[:n]
    genres_r_list=list(genres_counter)[:n]
    
#     print(time_r_list)
#     print(genres_r_list)

    search_answers_index=[]
    search_answers_number=0
    recommend_reason=-1
    
    
    
    #best: time and genres both pass
    for i in range(number_of_all_movies):
        if movie_time[i] in time_r_list and check_genres_in_list(movie_genres[i],genres_r_list) and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break
  
    if search_answers_index!=[]:
        recommend_reason=0
        return search_answers_index,recommend_reason
    

    
    #only considerate genres    
    for i in range(number_of_all_movies):
        if check_genres_in_list(movie_genres[i],genres_r_list) and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break

    if search_answers_index!=[]:
        recommend_reason=2
        return search_answers_index,recommend_reason    
    
    
    
    
    
    
    
    
    #only considerate time
    for i in range(number_of_all_movies):
        if movie_time[i] in time_r_list and movie_title[i] not in test:
            search_answers_number+=1
            search_answers_index.append(i)
            if search_answers_number==n:
                break

    if search_answers_index!=[]:
        recommend_reason=1
        return search_answers_index,recommend_reason

    else:
        print('no answers have been found')
    return search_answers_index,recommend_reason

In [63]:
recommend_movies(test,3)

([7, 9, 12], 0)

In [64]:
def recommend_answers(search_answers_outcomes):
    search_answers_index,recommend_reason=search_answers_outcomes
    movie_list=[movie_title[x] for x in search_answers_index]
    time_list=[movie_time[x] for x in search_answers_index]
    genres_list=[movie_genres[x] for x in search_answers_index]
    
    recommend_categories=''
    whether_found=(recommend_reason==-1)
    if recommend_reason==-1:
        recommend_categories='have not been found, so no reasons'
    elif recommend_reason==0:
        recommend_categories='all the movies have been published in the same year and the genres are the same'
    elif recommend_reason==1:
        recommend_categories='all the movies have been published in the same year'
    elif recommend_reason==2:
        recommend_categories='all the movies are the same genres'
    return movie_list,time_list,genres_list,recommend_categories,whether_found

In [65]:
recommend_answers(recommend_movies(test,3))

(['Tom and Huck', 'GoldenEye', 'Balto'],
 ['1995', '1995', '1995'],
 [['Adventure', 'Children'],
  ['Action', 'Adventure', 'Thriller'],
  ['Adventure', 'Animation', 'Children']],
 'all the movies have been published in the same year and the genres are the same',
 False)

In [66]:
def generate_answers_for_recommendation(search_answers_outcomes):
    movie_list,time_list,genres_list,recommend_categories,whether_found=search_answers_outcomes
    if whether_found==True:
        return 'no answers have been found, try again.'
    else:
        answer1='Adequate recommendations will be: {}, because {}.'.format(', '.join(movie_list),recommend_categories)
        answer2=' The details about the movies are shown as follows: '
        answer3=' publication date of each movie is {} and the genres of each movie is {}.'.format(', '.join(time_list),genres_list)
        return answer1+answer2+answer3
    

In [67]:
generate_answers_for_recommendation(recommend_answers(recommend_movies(test,3)))

"Adequate recommendations will be: Tom and Huck, GoldenEye, Balto, because all the movies have been published in the same year and the genres are the same. The details about the movies are shown as follows:  publication date of each movie is 1995, 1995, 1995 and the genres of each movie is [['Adventure', 'Children'], ['Action', 'Adventure', 'Thriller'], ['Adventure', 'Animation', 'Children']]."

In [68]:
test=['Who Am I (Kein System Ist Sicher)']

In [69]:
recommend_movies(test,3)

([8340, 8346, 8379], 0)

In [70]:
generate_answers_for_recommendation(recommend_answers(recommend_movies(test,3)))

"Adequate recommendations will be: Freezer, Jack Ryan: Shadow Recruit, Non-Stop, because all the movies have been published in the same year and the genres are the same. The details about the movies are shown as follows:  publication date of each movie is 2014, 2014, 2014 and the genres of each movie is [['Action', 'Thriller'], ['Action', 'Drama', 'Thriller', 'IMAX'], ['Action', 'Mystery', 'Thriller']]."

# KNN to recommend the movies

In [ ]:
ml_ratings = pd.read_csv('./ratings.csv', usecols=['userId', 'movieId', 'rating'])


# Multimedia

In [71]:
test='Julia Roberts'

In [72]:
class GraphOperations:
    """
    Handles all operations on the graph:
    - defines graph with fixed path that can be overwritten
    - load graph
    - query graph that returns the answer in a list
    """

    def __init__(self, graph_file="14_graph.nt"):
        self.graph = rdflib.Graph()
        self.load_graph(graph_file)

    def load_graph(self, graph_file):
        print("loading graph ...")
        self.graph.parse(graph_file, format="turtle")
        print("loaded graph successfully!")

    def query(self, message):
        message = message.replace('"""', "").replace("'''", "")
        message = str(message)
                
        temp = [str(s) for s, in self.graph.query(message)]
        # if temp!=[]:
        #     print('temp answer: ',temp)

        return temp

In [73]:
g=GraphOperations()

loading graph ...
loaded graph successfully!


In [74]:
def search_pictures(name):
    query =f"""
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?website WHERE{{    
            ?entity rdfs:label '{name}'@en.
            ?entity wdt:P18 ?website.
        }}
        LIMIT 1
    """
    search_website=g.query(query)
    if search_website==[]:
        return search_website,False
    else:
        return search_website,True

In [75]:
#we should considerate many images are generated simultaneously
#TODO:define a global count to name the images
image_number=0

In [76]:
def get_picture_as_answer(url_list_and_flag):
    search_website,whether_found=url_list_and_flag
    
    global image_number
    image_path=''
    
    if whether_found:
        def download_picture(url,image_number):
            
            html_page = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html_page)
            href=soup.find_all("div", {"class": "fullImageLink"})[0].find_all("a")[0].get("href")
            
            res=requests.get(href, stream = True)
            
            image_path=str(image_number)+'.jpg'
            image_number+=1
            
            with open(image_path,'wb') as f:
                shutil.copyfileobj(res.raw, f)        
            return image_path,image_number

        image_path,image_number=download_picture(search_website[0],image_number)
    return whether_found,image_path

In [77]:
def generate_answers_for_multimedia(whether_found_and_image_path):
    whether_found,image_path=whether_found_and_image_path
    if whether_found:
        print("we should send image here, but I haven't known how yet, but the image is downloaded in {}".format(image_path))
        return 'The image has been send!'
    return "I haven't found the image, sorry!"

In [78]:
generate_answers_for_multimedia(get_picture_as_answer(search_pictures(test)))

we should send image here, but I haven't known how yet, but the image is downloaded in 0.jpg


'The image has been send!'

# Crowdsourcing

In [79]:
test=['wd:Q11621', 'wdt:P2142']

In [89]:
crowd_df = pd.read_csv('./crowd_data.tsv', sep='\t')
# crowd_df.head()

In [90]:
normal_raters_map=(np.char.replace(crowd_df['LifetimeApprovalRate'].to_numpy(dtype=str),'%','').astype(int)>45)
crowd_df=crowd_df[normal_raters_map]
crowd_df=crowd_df[['HITId','Input1ID','Input2ID','Input3ID','AnswerLabel']].reset_index(drop=True)
crowd_df.head()

,HITId,Input1ID,Input2ID,Input3ID,AnswerLabel
0,1,wd:Q11621,wdt:P2142,792910554,CORRECT
1,1,wd:Q11621,wdt:P2142,792910554,INCORRECT
2,1,wd:Q11621,wdt:P2142,792910554,CORRECT
3,1,wd:Q11621,wdt:P2142,792910554,CORRECT
4,2,wd:Q603545,wdt:P2142,4300000,INCORRECT


In [91]:
def process_crowd(crowd_df):
    crowd_numpy=crowd_df.to_numpy().tolist()

    question_id=0
    rdf_turple=[]
    people_views=[]
    correct_number=0
    incorrect_number=0

    for survey_data in crowd_numpy:
        if survey_data[0]!=question_id:
            people_views.append([correct_number,incorrect_number])
            correct_number=0
            incorrect_number=0
            question_id+=1
            rdf_turple.append([survey_data[1],survey_data[2],survey_data[3]])
            if survey_data[4]=='CORRECT':
                correct_number+=1
            else:
                incorrect_number+=1
        else:
            if survey_data[4]=='CORRECT':
                correct_number+=1
            else:
                incorrect_number+=1


    people_views.append([correct_number,incorrect_number])
    people_views=people_views[1:]
    people_views=np.array(people_views,dtype=int)

    people_number=sum(people_views[0])
    evaluations_number=len(rdf_turple)

#     print(rdf_turple)
#     print(len(rdf_turple))
#     print(people_views)
#     print(len(people_views))
    
    
    
    
    # cal k in total

    p1=np.sum(people_views[:,0])/(people_number*evaluations_number)
    p2=np.sum(people_views[:,1])/(people_number*evaluations_number)
    P=(np.sum(np.square(people_views),axis=1)-people_number)/(people_number*(people_number-1))
    average_P=np.sum(P)/evaluations_number
    average_p_e=np.square(p1)+np.square(p2)
    total_k=(average_P-average_p_e)/(1-average_p_e)    
    
    
    return rdf_turple,people_views,total_k

In [92]:
def generate_answers_for_crowdsourcing(crowd_df,test):
    rdf_turple,people_views,total_k=process_crowd(crowd_df)
    
#     print(rdf_turple)
#     print(people_views)
    
    for i in range(len(rdf_turple)):
        if rdf_turple[i][0]==test[0] and rdf_turple[i][1]==test[1]:
            return rdf_turple[i][2],people_views[i],cal_k(people_views[i]),total_k

        
def cal_k(people_approvals):
    people_number=4
    evaluations_number=1
    p1=(people_approvals[0])/(people_number*evaluations_number)
    p2=(people_approvals[1])/(people_number*evaluations_number)
    P=(np.sum(np.square(people_approvals))-people_number)/(people_number*(people_number-1))

    average_P=P/evaluations_number
    average_p_e=np.square(p1)+np.square(p2)
        
    k=(average_P-average_p_e)/(1-average_p_e)
    return k

In [95]:
generate_answers_for_crowdsourcing(crowd_df,test)
#answer, people's views(3 for voting and 1 for rejecting), k for this question, k for the whole graph

('792910554', array([3, 1]), -0.3333333333333333, 0.00974025974025935)